In [ ]:
import sys
import os

# Make sure your project root is on PYTHONPATH
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from models.resnet50_backbone import ResNet50Backbone
from models.vgg_backbone import VGG19BNBackbone

from src.data_loader import ShanghaiTechDataset
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt



In [ ]:
# ====== Part A evaluation (ResNet-50 density-map backbone only) ======

# Ensure project root on PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Evaluating on device: {device}")

# Data transform and loader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

from src.data_loader import ShanghaiTechDataset
val_dataset_A = ShanghaiTechDataset(
    root="../data/ShanghaiTech",
    part="part_A",
    split="test_data",
    transform=transform,
    return_count=False
)
val_loader_A = DataLoader(val_dataset_A, batch_size=1, shuffle=False)

# Import only ResNet50 density backbone
from models.resnet50_backbone import ResNet50DensityBackbone

# Instantiate and load weights
model = ResNet50DensityBackbone(pretrained=False, upsample_to_input=True).to(device)
weight_path = "../models/part_A_resnet50_model.pth"
model.load_state_dict(torch.load(weight_path, map_location=device))
model.eval()

# Evaluate
total_mae = 0.0
total_mse = 0.0

with torch.no_grad():
    for img, gt_density in val_loader_A:
        img        = img.to(device)       # B×3×H×W
        gt_density = gt_density.to(device)  # B×1×H×W

        # True count
        gt_count = gt_density.sum(dim=(1,2,3))  # B

        # Predicted density and count
        pred_density = model(img)               # B×1×H×W
        pred_count   = pred_density.sum(dim=(1,2,3))

        # Accumulate
        total_mae += torch.abs(pred_count - gt_count).sum().item()
        total_mse += ((pred_count - gt_count)**2).sum().item()

N    = len(val_dataset_A)
mae  = total_mae / N
rmse = (total_mse / N)**0.5

print(f"Part A — resnet50: MAE = {mae:.3f}, RMSE = {rmse:.3f}")


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Part A — resnet50: MAE = 444.685, RMSE = 522.152


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to C:\Users\hp/.cache\torch\hub\checkpoints\vgg19_bn-c79401a0.pth


100%|██████████| 548M/548M [00:16<00:00, 35.9MB/s] 


RuntimeError: Error(s) in loading state_dict for VGG19BNBackbone:
	size mismatch for vgg.classifier.0.weight: copying a param with shape torch.Size([512, 25088]) from checkpoint, the shape in current model is torch.Size([512, 4096]).

In [3]:
# ====== Part B evaluation ======


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
val_dataset_B = ShanghaiTechDataset("../data/ShanghaiTech", part="part_B", split="test_data", transform=transform)
val_loader_B = DataLoader(val_dataset_B, batch_size=1)

for name, Cls in [("resnet50", ResNet50Backbone),
                  ("vgg19_bn", VGG19BNBackbone)]:
    model = Cls().to(device)
    model.load_state_dict(torch.load(f"../models/part_B_{name}_model.pth", map_location=device))
    model.eval()

    mae = mse = 0.0
    with torch.no_grad():
        for img, count_map in val_loader_B:
            img = img.to(device)
            count = count_map.sum(dim=(1,2)).unsqueeze(1).float().to(device)
            out = model(img)
            mae += torch.abs(out - count).item()
            mse += ((out - count)**2).item()

    mae /= len(val_dataset_B)
    rmse = (mse / len(val_dataset_B))**0.5
    print(f"Part B — {name}: MAE = {mae:.3f}, RMSE = {rmse:.3f}")

Part B — resnet50: MAE = 62.769, RMSE = 73.326


RuntimeError: Error(s) in loading state_dict for VGG19BNBackbone:
	size mismatch for vgg.classifier.0.weight: copying a param with shape torch.Size([512, 25088]) from checkpoint, the shape in current model is torch.Size([512, 4096]).